### Import libraries

In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from copy import deepcopy

### Read the sample data and see what it looks like

In [114]:
seqs = pd.read_csv("data/files-archive.tsv", sep = "\t")
seqs.head()

,study_id,specimen collector sample ID,sample collected by,sequence submitted by,sample collection date,sample collection date null reason,geo_loc_name (country),geo_loc_name (state/province/territory),organism,isolate,...,consensus sequence software name,consensus sequence software version,breadth of coverage value,depth of coverage value,reference genome accession,bioinformatics protocol,gene name,diagnostic pcr Ct value,diagnostic pcr Ct value null reason,GISAID accession
0,UHTC-ON,UHTC_0471,Unity Health Toronto,Ontario Institute for Cancer Research (OICR),2020-12-15,NaN,Canada,Ontario,Not Provided,hCoV-19/Canada/ON-UHTC_0471/2021,...,ARTIC-nanopolish,1.1.3,Not Provided,5384.4X,NaN,NaN,Not Provided,NaN,Not Provided,NaN
1,UHTC-ON,UHTC_0464,Unity Health Toronto,Ontario Institute for Cancer Research (OICR),2020-10-12,NaN,Canada,Ontario,Not Provided,hCoV-19/Canada/ON-UHTC_0464/2021,...,ARTIC-nanopolish,1.1.3,Not Provided,2912.4X,NaN,NaN,Not Provided,NaN,Not Provided,NaN
2,UHTC-ON,UHTC_0463,Unity Health Toronto,Ontario Institute for Cancer Research (OICR),2020-10-02,NaN,Canada,Ontario,Not Provided,hCoV-19/Canada/ON-UHTC_0463/2021,...,ARTIC-nanopolish,1.1.3,Not Provided,2661.6X,NaN,NaN,Not Provided,NaN,Not Provided,NaN
3,KHSC-ON,SLB3076,Queen's University / Kingston Health Sciences ...,Ontario Institute for Cancer Research (OICR),2020-04-12,NaN,Canada,Ontario,Not Provided,hCoV-19/Canada/ON-SLB3076/2021,...,ncov2019-artic-nf,OICR v1.6,Not Provided,1671.9X,NaN,NaN,Not Provided,NaN,Not Provided,NaN
4,KHSC-ON,SLB3121,Queen's University / Kingston Health Sciences ...,Ontario Institute for Cancer Research (OICR),2020-04-22,NaN,Canada,Ontario,Not Provided,hCoV-19/Canada/ON-SLB3121/2021,...,ncov2019-artic-nf,OICR v1.6,Not Provided,959.3X,NaN,NaN,Not Provided,NaN,Not Provided,NaN


### Print out all column names

In [115]:
print(seqs.columns)

Index(['study_id', 'specimen collector sample ID', 'sample collected by',
       'sequence submitted by', 'sample collection date',
       'sample collection date null reason', 'geo_loc_name (country)',
       'geo_loc_name (state/province/territory)', 'organism', 'isolate',
       'fasta header name', 'purpose of sampling',
       'purpose of sampling details', 'anatomical material', 'anatomical part',
       'body product', 'environmental material', 'environmental site',
       'collection device', 'collection method', 'host (scientific name)',
       'host disease', 'host age', 'host age null reason', 'host age unit',
       'host age bin', 'host gender', 'purpose of sequencing',
       'purpose of sequencing details', 'sequencing instrument',
       'sequencing protocol', 'raw sequence data processing method',
       'dehosting method', 'consensus sequence software name',
       'consensus sequence software version', 'breadth of coverage value',
       'depth of coverage value', 'r

### Remove the columns we don't need

In [116]:
seqs.drop(columns=['study_id', 'specimen collector sample ID', 'sample collected by',
       'sequence submitted by',
       'sample collection date null reason', 'geo_loc_name (country)',
       'organism', 'isolate',
       'fasta header name',
       'purpose of sampling details', 'anatomical material', 'anatomical part',
       'body product', 'environmental material', 'environmental site',
       'collection device', 'collection method', 'host (scientific name)',
       'host disease','host age null reason', 'host age unit',
       'purpose of sequencing details',
       'sequencing protocol', 'raw sequence data processing method',
       'dehosting method', 'consensus sequence software name',
       'consensus sequence software version', 'reference genome accession',
       'bioinformatics protocol', 'gene name', 
       'diagnostic pcr Ct value null reason'], inplace = True)

In [117]:
seqs.head()

,sample collection date,geo_loc_name (state/province/territory),purpose of sampling,host age,host age bin,host gender,purpose of sequencing,sequencing instrument,breadth of coverage value,depth of coverage value,diagnostic pcr Ct value,GISAID accession
0,2020-12-15,Ontario,Not Provided,NaN,Not Applicable,Not Provided,Not Provided,Oxford Nanopore,Not Provided,5384.4X,NaN,NaN
1,2020-10-12,Ontario,Not Provided,NaN,Not Applicable,Not Provided,Not Provided,Oxford Nanopore,Not Provided,2912.4X,NaN,NaN
2,2020-10-02,Ontario,Not Provided,NaN,Not Applicable,Not Provided,Not Provided,Oxford Nanopore,Not Provided,2661.6X,NaN,NaN
3,2020-04-12,Ontario,Not Provided,NaN,Not Applicable,Not Provided,Not Provided,Illumina,Not Provided,1671.9X,NaN,NaN
4,2020-04-22,Ontario,Not Provided,NaN,Not Applicable,Not Provided,Not Provided,Illumina,Not Provided,959.3X,NaN,NaN


### Let's replace the many forms of 'Not Provided' or NaN with np.nan and convert collection date to a date time object

In [118]:
seqs.replace({'Not Provided':np.nan, 'NaN':np.nan, 'Not Applicable':np.nan}, inplace = True)
seqs['sample collection date'] = pd.to_datetime(seqs['sample collection date'])
seqs.head()

,sample collection date,geo_loc_name (state/province/territory),purpose of sampling,host age,host age bin,host gender,purpose of sequencing,sequencing instrument,breadth of coverage value,depth of coverage value,diagnostic pcr Ct value,GISAID accession
0,2020-12-15,Ontario,NaN,NaN,NaN,NaN,NaN,Oxford Nanopore,NaN,5384.4X,NaN,NaN
1,2020-10-12,Ontario,NaN,NaN,NaN,NaN,NaN,Oxford Nanopore,NaN,2912.4X,NaN,NaN
2,2020-10-02,Ontario,NaN,NaN,NaN,NaN,NaN,Oxford Nanopore,NaN,2661.6X,NaN,NaN
3,2020-04-12,Ontario,NaN,NaN,NaN,NaN,NaN,Illumina,NaN,1671.9X,NaN,NaN
4,2020-04-22,Ontario,NaN,NaN,NaN,NaN,NaN,Illumina,NaN,959.3X,NaN,NaN


### What percent of the data is missing?

In [119]:
seqs.isna().sum()/len(seqs)*100

sample collection date                      0.000000
geo_loc_name (state/province/territory)     0.000000
purpose of sampling                        21.533453
host age                                   92.793075
host age bin                               54.541226
host gender                                56.600890
purpose of sequencing                      22.678132
sequencing instrument                       0.000000
breadth of coverage value                   0.105356
depth of coverage value                    25.508509
diagnostic pcr Ct value                    73.663829
GISAID accession                            1.722714
dtype: float64

### All samples have associated collection dates!
### Let's try grouping data by province to see if there's a pattern to what is missing.

In [120]:
seqs.columns = ['collection_date', 'province', 'sampling_purpose', 'age', 'age_bin', 'gender', 'sequencing_purpose', 'sequencing_instrument',
'coverage_breadth', 'coverage_depth', 'Ct', 'gisaid_accession']
seqs.groupby('province').count().rsub(seqs.groupby('province').size(), axis = 0)

,collection_date,sampling_purpose,age,age_bin,gender,sequencing_purpose,sequencing_instrument,coverage_breadth,coverage_depth,Ct,gisaid_accession
province,,,,,,,,,,,
Alberta,0,0,27740,27740,27740,0,0,0,26123,27740,1704
British Columbia,0,22576,34697,22576,22576,22576,0,0,0,34697,0
Manitoba,0,0,22,0,0,25,0,0,0,124,0
New Brunswick,0,0,507,16,5,16,0,0,0,90,0
Newfoundland and Labrador,0,0,4,0,0,0,0,0,0,0,0
Nova Scotia,0,0,1202,55,12,439,0,0,0,106,0
Ontario,0,111,29685,3229,5623,493,0,111,672,12022,111
Saskatchewan,0,0,3907,3847,3677,344,0,0,80,2831,0


### How many samples are there from each province?

In [121]:
seqs.groupby('province').size()

province
Alberta                      27740
British Columbia             34697
Manitoba                      1021
New Brunswick                  507
Newfoundland and Labrador      296
Nova Scotia                   1202
Ontario                      29685
Saskatchewan                 10209
dtype: int64

### How many complete values are there from each province in each field?

In [122]:
counts = seqs.groupby('province').count()
counts

,collection_date,sampling_purpose,age,age_bin,gender,sequencing_purpose,sequencing_instrument,coverage_breadth,coverage_depth,Ct,gisaid_accession
province,,,,,,,,,,,
Alberta,27740,27740,0,0,0,27740,27740,27740,1617,0,26036
British Columbia,34697,12121,0,12121,12121,12121,34697,34697,34697,0,34697
Manitoba,1021,1021,999,1021,1021,996,1021,1021,1021,897,1021
New Brunswick,507,507,0,491,502,491,507,507,507,417,507
Newfoundland and Labrador,296,296,292,296,296,296,296,296,296,296,296
Nova Scotia,1202,1202,0,1147,1190,763,1202,1202,1202,1096,1202
Ontario,29685,29574,0,26456,24062,29192,29685,29574,29013,17663,29574
Saskatchewan,10209,10209,6302,6362,6532,9865,10209,10209,10129,7378,10209


In [123]:
totals = counts['collection_date']

### What proportion of each field is complete for each province?

In [124]:
seqs_comp = deepcopy(pd.DataFrame(seqs))
seqs_comp = seqs_comp.groupby('province').count()

seqs_comp.divide(totals, axis = 0)

,collection_date,sampling_purpose,age,age_bin,gender,sequencing_purpose,sequencing_instrument,coverage_breadth,coverage_depth,Ct,gisaid_accession
province,,,,,,,,,,,
Alberta,1.0,1.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.000000,0.058291,0.000000,0.938572
British Columbia,1.0,0.349339,0.000000,0.349339,0.349339,0.349339,1.0,1.000000,1.000000,0.000000,1.000000
Manitoba,1.0,1.000000,0.978452,1.000000,1.000000,0.975514,1.0,1.000000,1.000000,0.878550,1.000000
New Brunswick,1.0,1.000000,0.000000,0.968442,0.990138,0.968442,1.0,1.000000,1.000000,0.822485,1.000000
Newfoundland and Labrador,1.0,1.000000,0.986486,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
Nova Scotia,1.0,1.000000,0.000000,0.954243,0.990017,0.634775,1.0,1.000000,1.000000,0.911814,1.000000
Ontario,1.0,0.996261,0.000000,0.891225,0.810578,0.983392,1.0,0.996261,0.977362,0.595014,0.996261
Saskatchewan,1.0,1.000000,0.617298,0.623176,0.639828,0.966304,1.0,1.000000,0.992164,0.722696,1.000000


In [125]:
seqs.head()

,collection_date,province,sampling_purpose,age,age_bin,gender,sequencing_purpose,sequencing_instrument,coverage_breadth,coverage_depth,Ct,gisaid_accession
0,2020-12-15,Ontario,NaN,NaN,NaN,NaN,NaN,Oxford Nanopore,NaN,5384.4X,NaN,NaN
1,2020-10-12,Ontario,NaN,NaN,NaN,NaN,NaN,Oxford Nanopore,NaN,2912.4X,NaN,NaN
2,2020-10-02,Ontario,NaN,NaN,NaN,NaN,NaN,Oxford Nanopore,NaN,2661.6X,NaN,NaN
3,2020-04-12,Ontario,NaN,NaN,NaN,NaN,NaN,Illumina,NaN,1671.9X,NaN,NaN
4,2020-04-22,Ontario,NaN,NaN,NaN,NaN,NaN,Illumina,NaN,959.3X,NaN,NaN
